In [ ]:
%%html
<style>div.input {display:none;} div.output_stderr{display:none}</style>

In [ ]:
#                                        DEVELOPMENT library list:
import warnings
warnings.filterwarnings('ignore')

import os
import sys

import pandas as pd
from pandas.io.common import EmptyDataError
import numpy as np

from IPython.display import display, HTML
import ipywidgets as widgets
import traitlets

from knpackage import toolbox as kn

sys.path.insert(1, './src')
import mini_pipelines_toolbox as mpt

#                                             Define the required directory names structure
USER_BASE_DIRECTORY = os.getcwd()
USER_DATA_DIRECTORY = os.path.join(USER_BASE_DIRECTORY, 'user_data')
USER_RESULTS_DIRECTORY = os.path.join(USER_BASE_DIRECTORY, 'results')

USER_BASE_DIRECTORIES_LIST = []
list_dir_list = os.listdir(USER_BASE_DIRECTORY)
for maybe_dir in list_dir_list:
    maybe_dir_full = os.path.join(USER_BASE_DIRECTORY, maybe_dir)
    if os.path.isdir(maybe_dir_full):
        USER_BASE_DIRECTORIES_LIST.append(maybe_dir_full)

#                                             Create user_data and results directories if they DNE
if len(USER_BASE_DIRECTORIES_LIST) < 1 or not USER_DATA_DIRECTORY in USER_BASE_DIRECTORIES_LIST:
    os.mkdir(USER_DATA_DIRECTORY)
    
if len(USER_BASE_DIRECTORIES_LIST) < 1 or not USER_RESULTS_DIRECTORY in USER_BASE_DIRECTORIES_LIST:
    os.mkdir(USER_RESULTS_DIRECTORY)


def user_data_list(target_dir, FEXT):
    """ user_file_list = update_user_data_list(user_data_dir, FEXT)
    Args:
        target_dir:     directory to list
        FEXT:           File extension list e.g. ['.tsv', '.txt']
    """
    my_file_list = []
    for f in os.listdir(target_dir):
        if os.path.isfile(os.path.join(target_dir, f)):
            noNeed, f_ext = os.path.splitext(f)
            if f_ext in FEXT:
                my_file_list.append(f)
                
    if len(my_file_list) <= 0:
        my_file_list.append('No Data')
        
    return my_file_list

#                                             define Alias
results_dir = USER_RESULTS_DIRECTORY
input_data_dir = USER_DATA_DIRECTORY

USER_DATAFILE_EXTENSIONS_LIST = ['.tsv', '.txt', '.df', '.gz']


def get_dropdown_files_listbox():
    """ user_data dropdown listbox - for use as master listbox | repeated display listbox 
    
    Usage - set new listbox linkage:
        flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), (flistbx, 'options'))
    Usage - for display in multiple cells:
        files_dropdown_main = get_dropdown_files_listbox()
    Usage - display:
        display(files_dropdown_main)
        layout=widgets.Layout(width='80%', height='80px'),
    """
    files_dropdown_stock_box = widgets.Dropdown(
        options=user_data_list(input_data_dir, USER_DATAFILE_EXTENSIONS_LIST),
        value=user_data_list(input_data_dir, USER_DATAFILE_EXTENSIONS_LIST)[0],
        description='user_data:'
    )
    return files_dropdown_stock_box
files_dropdown_main = get_dropdown_files_listbox()


def update_user_data(button):
    """ update files list in master (invisible) files dropdown list -- which is linked to all 
    Usage - display:
        display(update_user_data_button)
    Usage - update all main listbox children:
        push the button or -- update_user_data(update_user_data_button)
    """
    files_dropdown_main.options = user_data_list(input_data_dir, USER_DATAFILE_EXTENSIONS_LIST)
    files_dropdown_main.value=user_data_list(input_data_dir, USER_DATAFILE_EXTENSIONS_LIST)[0]

update_user_data_button = widgets.Button(description='Udate user_data', user_data_list='No Data')
update_user_data_button.on_click(update_user_data)

def get_view_box():
    """ each view button needs a named HTML display area """
    vb = widgets.HTML(
        value="",
        description="")
    return vb

def visualize_selected_file(button):
    """ callback for buttons with 'fname_list' OR a dropdown listbox with 'file_selector' - display file upper left.
    Args:
        button:         an ipywidgets.Button object with an ipywidgets.Dropdown (.file_selector) object containing 
                        the selected file name as its .value field.
    """
    try:
        if hasattr(button, 'fname_list') == True: 
            full_fname_list = button.fname_list
            S = ''
            for full_fname in full_fname_list: 
                df = pd.read_csv(full_fname,sep='\t',header=0,index_col=0)
                path_not, f_name = os.path.split(full_fname)
                Step = df.iloc[0:10,0:10];
                S = S + str(df.shape) + '    ' + f_name + Step.to_html()
            button.view_box.value = S
        else: 
            full_fname = os.path.join(input_data_dir, button.file_selector.value)
            df = pd.read_csv(full_fname, sep='\t', header=0, index_col=0)
            Step = df.iloc[0:10,0:10];
            button.view_box.value = Step.to_html()
            button.view_box.description=str(df.shape)
        
    except OSError:
        print("No input data! ")
        
    except EmptyDataError:
        print("Empty input data! ")
        
    except:
        print("Invalid input data again! ")

## Spreadsheet Transpose

In [ ]:
def transpose_selected_file(button):
    """ read, transpose and write the new spreadsheet using the Spreadsheets_Transformation - mini_pipelines_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with an ipywidgets.Dropdown (.file_selector) object containing 
                        the selected file name as its .value field.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_name_full_path'] = os.path.join(input_data_dir, button.file_selector.value)
    mpt.run_transpose(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

#                           spreadsheet dropdown listbox:
transpose_flistbx = get_dropdown_files_listbox()
transpose_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), (transpose_flistbx, 'options'))

#                           spreadsheet view button:
visualize_file_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
)
visualize_file_button.view_box = get_view_box()
visualize_file_button.file_selector = transpose_flistbx
visualize_file_button.on_click(visualize_selected_file)

#                           display the controls:
display(update_user_data_button)
display(transpose_flistbx)
display(visualize_file_button)
display(visualize_file_button.view_box)

In [ ]:
#                           execute transpose - write button:
transpose_execute_button = widgets.Button(
    description='Transpose',
    disabled=False,
    button_style='',
    tooltip='transpose selected file',
    )
transpose_execute_button.view_box = get_view_box()
transpose_execute_button.file_selector = transpose_flistbx
transpose_execute_button.on_click(transpose_selected_file)

display(transpose_execute_button)
display(transpose_execute_button.view_box)

## Spreadsheet 1, Spreadsheet 2, Common Samples (two Spreadsheets)

In [ ]:
def get_common_samples(button):
    """ local function to read 2 spreadsheet files, find and write the common samples to 2 new spreadsheets
        using the Spreadsheets_Transformation - mini_pipelines_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.

    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_1_file_name'] = os.path.join(input_data_dir, button.file_1_selector.value)
    run_parameters['spreadsheet_2_file_name'] = os.path.join(input_data_dir, button.file_2_selector.value)
    mpt.run_common_samples_df(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)


#                           first dropdown listbox and visualize button:
common_samples_flistbx_1 = get_dropdown_files_listbox()
common_samples_flistbx_1_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                 (common_samples_flistbx_1, 'options'))

common_samples_flistbx_1_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
common_samples_flistbx_1_view_button.file_selector=common_samples_flistbx_1
common_samples_flistbx_1_view_button.on_click(visualize_selected_file)
common_samples_flistbx_1_view_button.view_box = get_view_box()

display(update_user_data_button)
display(common_samples_flistbx_1)
display(common_samples_flistbx_1_view_button)
display(common_samples_flistbx_1_view_button.view_box)

In [ ]:
#                           second dropdown listbox and visualize button:
common_samples_flistbx_2 = get_dropdown_files_listbox()
common_samples_flistbx_2_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                 (common_samples_flistbx_2, 'options'))

common_samples_flistbx_2_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
common_samples_flistbx_2_view_button.file_selector=common_samples_flistbx_2
common_samples_flistbx_2_view_button.on_click(visualize_selected_file)
common_samples_flistbx_2_view_button.view_box = get_view_box()

display(common_samples_flistbx_2)
display(common_samples_flistbx_2_view_button)
display(common_samples_flistbx_2_view_button.view_box)

In [ ]:
#                           common samples execute button:
common_samples_execute_button = widgets.Button(
    description='Get Common Samples',
    disabled=False,
    button_style='',
    tooltip='get common samples button',
    )
common_samples_execute_button.file_1_selector=common_samples_flistbx_1
common_samples_execute_button.file_2_selector=common_samples_flistbx_2
common_samples_execute_button.view_box = get_view_box()

common_samples_execute_button.on_click(get_common_samples)
display(common_samples_execute_button)
display(common_samples_execute_button.view_box)

## Spreadsheet 1, Spreadsheet 2,  Merge to one Spreadsheet

In [ ]:
def merge_spreadsheets(button):
    """ Union: local function to read 2 spreadsheet files, find and write the common samples to 1 new spreadsheets
        using the Spreadsheets_Transformation - mini_pipelines_toolbox
        
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_1_file_name'] = os.path.join(input_data_dir, button.file_1_selector.value)
    run_parameters['spreadsheet_2_file_name'] = os.path.join(input_data_dir, button.file_2_selector.value)
    mpt.run_merge_df(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

#                           first dropdown listbox and visualize button:
merge_flistbx_1 = get_dropdown_files_listbox()
merge_flistbx_1_dlink = traitlets.dlink((files_dropdown_main, 'options'), (merge_flistbx_1, 'options'))

merge_flistbx_1_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
merge_flistbx_1_view_button.file_selector=merge_flistbx_1
merge_flistbx_1_view_button.on_click(visualize_selected_file)
merge_flistbx_1_view_button.view_box = get_view_box()

display(update_user_data_button)
display(merge_flistbx_1)
display(merge_flistbx_1_view_button)
display(merge_flistbx_1_view_button.view_box)

In [ ]:
#                           second dropdown listbox and visualize button:
merge_flistbx_2 = get_dropdown_files_listbox()
merge_flistbx_2_dlink = traitlets.dlink((files_dropdown_main, 'options'), (merge_flistbx_2, 'options'))

merge_flistbx_2_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
merge_flistbx_2_view_button.file_selector=merge_flistbx_2
merge_flistbx_2_view_button.on_click(visualize_selected_file)
merge_flistbx_2_view_button.view_box = get_view_box()

display(merge_flistbx_2)
display(merge_flistbx_2_view_button)
display(merge_flistbx_2_view_button.view_box)

In [ ]:
#                           merge files execute button:
merge_execute_button = widgets.Button(
    description='Merge Spreadsheets',
    disabled=False,
    button_style='',
    tooltip='merge spreadsheets',
    )
merge_execute_button.file_1_selector=merge_flistbx_1
merge_execute_button.file_2_selector=merge_flistbx_2
merge_execute_button.view_box = get_view_box()

merge_execute_button.on_click(merge_spreadsheets)
display(merge_execute_button)
display(merge_execute_button.view_box)

## Spreadsheet Select Genes from list

In [ ]:
def read_a_list_file(input_file_name):
    """ local function to read a list from a text file
    
    Args:
        input_file_name:    full path name of a file containing a list
    
    Returns:
        string_input:       the list that was contained in the file
    """
    with open(input_file_name, 'r') as fh:
        str_input = fh.read()
    return list(str_input.split())

def select_genes(button):
    """ local function to read files and get common samples and write 
        using the Spreadsheets_Transformation - mini_pipelines_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_file_name'] = os.path.join(input_data_dir, button.file_1_selector.value)
    run_parameters['gene_list_file_name'] = os.path.join(input_data_dir, button.file_2_selector.value)
    mpt.run_select_genes(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

#                           spreadsheet select dropdown listbox and visualize button:
select_spreadsheet_flistbx = get_dropdown_files_listbox()
select_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (select_spreadsheet_flistbx, 'options'))
select_spreadsheet_flistbx.description='Spreadsheet:'

select_spreadsheet_flistbx_1_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
select_spreadsheet_flistbx_1_view_button.file_selector=select_spreadsheet_flistbx
select_spreadsheet_flistbx_1_view_button.on_click(visualize_selected_file)
select_spreadsheet_flistbx_1_view_button.view_box = get_view_box()

display(update_user_data_button)
display(select_spreadsheet_flistbx)
display(select_spreadsheet_flistbx_1_view_button)
display(select_spreadsheet_flistbx_1_view_button.view_box)

In [ ]:
#                           "list of genes" file selector dropdown listbox and visualize button:
select_listfile_flistbx = get_dropdown_files_listbox()
select_listfile_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                  (select_listfile_flistbx, 'options'))
select_listfile_flistbx.description='list-file:'

select_listfile_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
select_listfile_flistbx_view_button.file_selector=select_listfile_flistbx
select_listfile_flistbx_view_button.on_click(visualize_selected_file)
select_listfile_flistbx_view_button.view_box = get_view_box()

display(select_listfile_flistbx)
display(select_listfile_flistbx_view_button)
display(select_listfile_flistbx_view_button.view_box)

In [ ]:
#                           select genes execute button:
select_genes_execute_button = widgets.Button(
    description='select genes',
    disabled=False,
    button_style='',
    tooltip='select genes from list',
    )
select_genes_execute_button.file_1_selector=select_spreadsheet_flistbx
select_genes_execute_button.file_2_selector=select_listfile_flistbx
select_genes_execute_button.view_box = get_view_box()

select_genes_execute_button.on_click(select_genes)
display(select_genes_execute_button)
display(select_genes_execute_button.view_box)

##  Spreadsheet, Cluster Labels, Averages

In [ ]:
def get_cluster_averages(button):
    """ local function to get the averages for the dictionay selected samples in a spreadsheet
        using the Spreadsheets_Transformation - mini_pipelines_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_file_name'] = os.path.join(input_data_dir, button.file_1_selector.value)
    run_parameters['sample_labels_file_name'] = os.path.join(input_data_dir, button.file_2_selector.value)
    run_parameters['centroid_statistic'] = button.centroid_selector.value
    
    mpt.run_cluster_statistics_df(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

#                           spreadsheet select dropdown listbox and visualize button:
cluster_averages_spreadsheet_flistbx = get_dropdown_files_listbox()
cluster_averages_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (cluster_averages_spreadsheet_flistbx, 'options'))
cluster_averages_spreadsheet_flistbx.description='Spreadsheet:'

cluster_averages_spreadsheet_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
cluster_averages_spreadsheet_flistbx_view_button.file_selector=cluster_averages_spreadsheet_flistbx
cluster_averages_spreadsheet_flistbx_view_button.on_click(visualize_selected_file)
cluster_averages_spreadsheet_flistbx_view_button.view_box = get_view_box()

display(update_user_data_button)
display(cluster_averages_spreadsheet_flistbx)
display(cluster_averages_spreadsheet_flistbx_view_button)
display(cluster_averages_spreadsheet_flistbx_view_button.view_box)

In [ ]:
#                           list of genes file select dropdown listbox and visualize button:
select_clusters_flistbx = get_dropdown_files_listbox()
select_clusters_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                  (select_clusters_flistbx, 'options'))
select_clusters_flistbx.description='dictionary file:'

select_clusters_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
select_clusters_flistbx_view_button.file_selector=select_clusters_flistbx
select_clusters_flistbx_view_button.on_click(visualize_selected_file)
select_clusters_flistbx_view_button.view_box = get_view_box()

display(select_clusters_flistbx)
display(select_clusters_flistbx_view_button)
display(select_clusters_flistbx_view_button.view_box)

In [ ]:
# centroid_selector_button
AVAILABLE_STATISTICAL_CENTROIDS = ['std', 'median', 'mean']
centroid_selector_dropdown = widgets.Dropdown(
    options=AVAILABLE_STATISTICAL_CENTROIDS,
    value=AVAILABLE_STATISTICAL_CENTROIDS[0],
    description='Statistic:'
)
display(centroid_selector_dropdown)

In [ ]:
#                           get_cluster_averages files execute button:
cluster_averages_execute_button = widgets.Button(
    description='Calculate Averages',
    disabled=False,
    button_style='',
    tooltip='get cluster averages',
)
cluster_averages_execute_button.file_1_selector=cluster_averages_spreadsheet_flistbx
cluster_averages_execute_button.file_2_selector=select_clusters_flistbx
cluster_averages_execute_button.centroid_selector=centroid_selector_dropdown
cluster_averages_execute_button.view_box = get_view_box()
    
cluster_averages_execute_button.on_click(get_cluster_averages)
display(cluster_averages_execute_button)
display(cluster_averages_execute_button.view_box)

## Spreadsheet, Phenotype: Select Categorical

In [ ]:
def select_categorical(button):
    """ local function to read files and select categorical and write 
        using the Spreadsheets_Transformation - mini_pipelines_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_file_name'] = os.path.join(input_data_dir, button.spreadsheet_file_selector.value)
    run_parameters['phenotype_file_name'] = os.path.join(input_data_dir, button.phenotype_file_selector.value)
    run_parameters['phenotype_id'] = button.phenotype_id_selector.value
    run_parameters['select_category'] = button.category_selectory.value
    
    mpt.run_select_subtype_df(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

#                           spreadsheet select dropdown listbox and visualize button:
select_categorical_spreadsheet_flistbx = get_dropdown_files_listbox()
select_categorical_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (select_categorical_spreadsheet_flistbx, 'options'))
select_categorical_spreadsheet_flistbx.description='Spreadsheet:'

select_categorical_spreadsheet_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    )
select_categorical_spreadsheet_flistbx_view_button.file_selector=select_categorical_spreadsheet_flistbx
select_categorical_spreadsheet_flistbx_view_button.on_click(visualize_selected_file)
select_categorical_spreadsheet_flistbx_view_button.view_box = get_view_box()

display(update_user_data_button)
display(select_categorical_spreadsheet_flistbx)
display(select_categorical_spreadsheet_flistbx_view_button)
display(select_categorical_spreadsheet_flistbx_view_button.view_box)

In [ ]:
#                           phenotype file select dropdown listbox and visualize button:
select_phenotype_flistbx = get_dropdown_files_listbox()
select_phenotype_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                  (select_phenotype_flistbx, 'options'))
select_phenotype_flistbx.description='phenotype:'

select_phenotype_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    )
select_phenotype_flistbx_view_button.file_selector=select_phenotype_flistbx
select_phenotype_flistbx_view_button.on_click(visualize_selected_file)
select_phenotype_flistbx_view_button.view_box = get_view_box()

display(select_phenotype_flistbx)
display(select_phenotype_flistbx_view_button)
display(select_phenotype_flistbx_view_button.view_box)

In [ ]:
# Create two special dropdown listboxes S.T. 
#       - the first shows the list of phenotype ids in the phenotype file
#       - the second shows the unique values in the selected phenotype id
def reset_phenotype_id_list(change):
    """ update the phenotype id options and select the first one in phenotype_id_listbox, and...
        update the category_select_listbox options and select the first
    """
    new_file = os.path.join(input_data_dir, change.new)
    new_df = pd.read_csv(new_file, sep='\t', index_col=0, header=0)
    
    new_phenotpye_list = new_df.columns.tolist()
    phenotype_id_listbox.options = new_phenotpye_list
    
    phenotype_id_listbox.value = new_phenotpye_list[0]
    
    a_list = new_df[phenotype_id_listbox.value]
    a_list = a_list.dropna()
    category_select_listbox.options = sorted(set(a_list))
    category_select_listbox.value = category_select_listbox.options[0]

phenotype_id_listbox = widgets.Dropdown(
    options=[''],
    value='',
    description='pheontype id:',
)
select_phenotype_flistbx.target_dir=input_data_dir
#       - phenotype file selection -- change this listbox
select_phenotype_flistbx.observe(reset_phenotype_id_list, names='value')

def reset_category_list(change):
    """ update the category_select_listbox options and select the first option
    """
    new_file = os.path.join(input_data_dir, select_phenotype_flistbx.value)
    new_df = pd.read_csv(new_file, sep='\t', index_col=0, header=0)
    
    a_list = new_df[phenotype_id_listbox.value]
    a_list = a_list.dropna()
    
    category_select_listbox.options = sorted(set(a_list))
    category_select_listbox.value = category_select_listbox.options[0]

category_select_listbox = widgets.Dropdown(
    options=[''],
    value='',
    description='category:'
)
#       - phenotype id change change this listbox
phenotype_id_listbox.observe(reset_category_list, names='value')

display(phenotype_id_listbox)
display(category_select_listbox)

In [ ]:
#                           get_cluster_averages files execute button:
select_categorical_execute_button = widgets.Button(
    description='Select Categorical',
    disabled=False,
    button_style='',
    tooltip='select category in both',
)
select_categorical_execute_button.spreadsheet_file_selector=select_categorical_spreadsheet_flistbx
select_categorical_execute_button.phenotype_file_selector=select_phenotype_flistbx
select_categorical_execute_button.phenotype_id_selector=phenotype_id_listbox
select_categorical_execute_button.category_selectory=category_select_listbox
select_categorical_execute_button.view_box = get_view_box()

select_categorical_execute_button.on_click(select_categorical)
display(select_categorical_execute_button)
display(select_categorical_execute_button.view_box)

## Spreadsheet numerical transformations

In [ ]:
def select_numerical_transform(button):
    """ local function to read files and select categorical and write 
        using the Spreadsheets_Transformation - mini_pipelines_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_name_full_path'] = os.path.join(input_data_dir, button.spreadsheet_file_selector.value)
    run_parameters['numeric_function'] = button.numerical_function.value
    if button.numerical_function.value == 'z_transform':
        run_parameters['z_transform_axis'] = button.numerical_function.z_transform_axis.value
        run_parameters['z_transform_ddof'] = button.numerical_function.z_transform_ddof.value
        
    elif button.numerical_function.value == 'log_transform':
        run_parameters['log_transform_log_base'] = button.numerical_function.log_transform_log_base.value
        run_parameters['log_transform_log_offset'] = button.numerical_function.log_transform_log_offset.value
        
    elif button.numerical_function.value == 'threshold':
        run_parameters['threshold_cut_off'] = button.numerical_function.threshold_cut_off.value
        run_parameters['threshold_substitution_value'] = button.numerical_function.threshold_substitution_value.value
        run_parameters['threshold_scope'] = button.numerical_function.threshold_scope.value
    
    mpt.run_spreadsheet_numerical_transform(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

#                           spreadsheet select dropdown listbox and visualize button:
select_numerical_spreadsheet_flistbx = get_dropdown_files_listbox()
select_numerical_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (select_numerical_spreadsheet_flistbx, 'options'))
select_numerical_spreadsheet_flistbx.description='Spreadsheet:'

select_numerical_spreadsheet_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    )
select_numerical_spreadsheet_flistbx_view_button.file_selector=select_numerical_spreadsheet_flistbx
select_numerical_spreadsheet_flistbx_view_button.on_click(visualize_selected_file)
select_numerical_spreadsheet_flistbx_view_button.view_box = get_view_box()

display(update_user_data_button)
display(select_numerical_spreadsheet_flistbx)
display(select_numerical_spreadsheet_flistbx_view_button)
display(select_numerical_spreadsheet_flistbx_view_button.view_box)

In [ ]:
""" display a numeric function options dialog and if selected is:
    z_transform     - show radio button row or columns, and a int text box for ddof
    log_transform   - show log_transform float base text box, and a float offset box
    threshold       - show a float threshold cut-off, substitution box, and threshold bound [SUB_BELOW, SUB_ABOVE]
"""
def reset_aux_controls(change):
    """ callback - sub controls setup for numerical_transform method options """
    threshold_scope = ['SUB_BELOW', 'SUB_ABOVE']
    log_trans_base = np.exp(1)
    default_threshold = 0.5
    
    if change.old == 'z_transform':
        numerical_function_dropdown.z_transform_axis.close()
        numerical_function_dropdown.z_transform_ddof.close()
        
    elif change.old == 'log_transform':
        numerical_function_dropdown.log_transform_log_base.close()
        numerical_function_dropdown.log_transform_log_offset.close()
        
    elif change.old == 'threshold':
        numerical_function_dropdown.threshold_cut_off.close()
        numerical_function_dropdown.threshold_substitution_value.close()
        numerical_function_dropdown.threshold_scope.close()
    
    if change.new == 'abs':
        pass
    
    elif change.new == 'z_transform':
        numerical_function_dropdown.z_transform_axis = widgets.RadioButtons(options={'rows': 0, 'columns': 1},
                                                                            value=0, description='axis', 
                                                                            disabled=False)
        numerical_function_dropdown.z_transform_ddof = widgets.IntText(value=0,description='ddof',disabled=False)
        display(numerical_function_dropdown.z_transform_axis)
        display(numerical_function_dropdown.z_transform_ddof)
        
    elif change.new == 'log_transform':
        numerical_function_dropdown.log_transform_log_base = widgets.FloatText(value=log_trans_base,
                                                                               description='log_base', disabled=False)
        numerical_function_dropdown.log_transform_log_offset = widgets.FloatText(value=0,
                                                                               description='log_base', disabled=False)
        display(numerical_function_dropdown.log_transform_log_base)
        display(numerical_function_dropdown.log_transform_log_offset)
        
    elif change.new == 'threshold':
        numerical_function_dropdown.threshold_cut_off = widgets.FloatText(value=default_threshold,
                                                                         description='cut-off', disabled=False)
        numerical_function_dropdown.threshold_substitution_value = widgets.FloatText(value=0,
                                                                         description='substitute', disabled=False)
        numerical_function_dropdown.threshold_scope = widgets.RadioButtons(options=threshold_scope,
                                                                          value=threshold_scope[0],
                                                                           description='sub', disabled=False)
        
        display(numerical_function_dropdown.threshold_cut_off)
        display(numerical_function_dropdown.threshold_substitution_value)
        display(numerical_function_dropdown.threshold_scope)
        
numerical_function_options = ['abs', 'z_transform', 'log_transform', 'threshold']
numerical_function_dropdown = widgets.Dropdown(
    options=numerical_function_options,
    value=numerical_function_options[0],
    description='stats function',
)
numerical_function_dropdown.observe(reset_aux_controls, names='value')
display(numerical_function_dropdown)

In [ ]:
#                           get numerical_measure execute button:
get_numerical_function_execute_button = widgets.Button(
    description='calculate',
    disabled=False,
    button_style='',
    tooltip='run and write selections',
    )
get_numerical_function_execute_button.spreadsheet_file_selector=select_numerical_spreadsheet_flistbx
get_numerical_function_execute_button.numerical_function=numerical_function_dropdown
get_numerical_function_execute_button.on_click(select_numerical_transform)
get_numerical_function_execute_button.view_box = get_view_box()

display(get_numerical_function_execute_button)
display(get_numerical_function_execute_button.view_box)

## Spreadsheet genes or samples statistics 

In [ ]:
def get_stats_value(button):
    """ local function to read spreadsheet file, select statistical measure and write 
        using the Spreadsheets_Transformation - mini_pipelines_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_file_name'] = os.path.join(input_data_dir, button.spreadsheet_file_selector.value)
    run_parameters['stats_function'] = button.stats_function.value
    run_parameters['direction_reference'] = button.direction_reference.value
    
    mpt.run_stats_df(run_parameters)
    
    #                       reinitialize files list in user_data dropdown listboxes and display result file (s)
    update_user_data(update_user_data_button)
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

#                           spreadsheet select dropdown listbox and visualize button:
stats_function_spreadsheet_flistbx = get_dropdown_files_listbox()
stats_function_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (stats_function_spreadsheet_flistbx, 'options'))
stats_function_spreadsheet_flistbx.description='Spreadsheet:'

stats_function_spreadsheet_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    )
stats_function_spreadsheet_flistbx_view_button.file_selector=stats_function_spreadsheet_flistbx
stats_function_spreadsheet_flistbx_view_button.on_click(visualize_selected_file)
stats_function_spreadsheet_flistbx_view_button.view_box = get_view_box()

display(update_user_data_button)
display(stats_function_spreadsheet_flistbx)
display(stats_function_spreadsheet_flistbx_view_button)
display(stats_function_spreadsheet_flistbx_view_button.view_box)

In [ ]:
stats_function_options = ['min', 'max', 'mean', 'median', 'variation', 'std_deviation', 'sum']
stats_function_dropdown = widgets.Dropdown(
    options=stats_function_options,
    value=stats_function_options[0],
    description='stats function'
)
display(stats_function_dropdown)

direction_reference_options = ['columns', 'rows', 'all']
direction_reference_dropdown = widgets.Dropdown(
    options=direction_reference_options,
    value=direction_reference_options[0],
    description='direction'
)
display(direction_reference_dropdown)

In [ ]:
#                           get stats execute button:
get_stats_execute_button = widgets.Button(
    description='calculate',
    disabled=False,
    button_style='',
    tooltip='run and write selections',
    )
get_stats_execute_button.spreadsheet_file_selector=stats_function_spreadsheet_flistbx
get_stats_execute_button.stats_function=stats_function_dropdown
get_stats_execute_button.direction_reference=direction_reference_dropdown
get_stats_execute_button.on_click(get_stats_value)
get_stats_execute_button.view_box = get_view_box()

display(get_stats_execute_button)
display(get_stats_execute_button.view_box)